# Tuning the Top-k Query parameter

First we set up the environment

In [8]:
from dotenv import load_dotenv
import os
import pandas as pd
from time import time
import torch
from TextEnrichers import get_enricher, TextEnricher
from database.database import Database
from Embedders import Embedder, get_embedder
from tqdm import tqdm

load_dotenv('.env', override=True)
print(os.getenv('DB_PORT'))


5432


In [9]:
# Database setup
db = Database()
db.test_connection()

device = 'cuda' if torch.cuda.is_available(
) else 'mps' if torch.mps.is_available() else 'cpu'
print(f"Using device: {device}")

=================================CONFIG=================================
Database         User             Host                             Port            
citeline_db      bbasseri         localhost                        5432            
Database version: ('PostgreSQL 17.3 (Homebrew) on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)
Using device: mps


## Investigating precision over `k`

For our various embedding models and enrichment strategies, we want to know the smallest `top_k` value that will still retrieve the target reference for a given sentence. 

To investigate this, we'll sample 100 examples from the non-trivial training data. Each example typically has 1-2 target DOI's. For each example, we'll query the database with a large `top_k` parameter to start, so we can be sure the database returns the target references. Then we can ask at what index in the query results does a target DOI first appear. Ideally, the ranks will all be very high, indicated by having *low* indices in the query results. We also expect enriched examples to have their target doi's higher ranked (lower indices).

In [10]:
examples = pd.read_json('data/dataset/100/nontrivial.jsonl', lines=True)


def lowest_index_matching_doi(target_doi: str, query_results: list) -> int:
    """
    Returns the first index of the query results where the chunk doi matches the target doi.
    If no match is found, returns -1.
    """
    for i, result in enumerate(query_results):
        if target_doi == result.doi:
            return i
    return -1


def get_ranks(
    example: pd.Series,
    embedding,
    top_k: int,
    probes: int,
    table_name: str = 'lib',
    target_column: str = 'bge',
) -> list[int]:
    
    target_dois = example['citation_dois']

    # Query
    start = time()
    query_results = db.query_vector_column(
        query_vector=embedding,
        target_column=target_column,
        table_name=table_name,
        pubdate=example['pubdate'],
        top_k=top_k,
        use_index=True,
        probes=probes,
    )
    print(f"Query time: {time() - start:.2f}s")
    print(f"Found {len(query_results)} results")
    
    ranks = [lowest_index_matching_doi(
        target_doi=doi, query_results=query_results) for doi in target_dois]
    return ranks


def ranks_at_k(
        examples: pd.DataFrame,
        embedder_name: str,
        enricher_name: str,
        target_column: str,
        top_k: int,
        probes: int=256) -> list[int]:
    """
    Calculates the 'ranks' for a given embedding model and enrichment function.
    The ranks are the indices of the first chunk with the target doi for each example, i.e.
    the lowest k that would retrieve a chunk with the target doi.
    """

    # Setup
    print(f"Embedding model: {embedder_name}, Enricher: {enricher_name}")
    embedder = get_embedder(embedder_name, device=device)
    enricher = get_enricher(
        enricher_name, path_to_data='data/preprocessed/reviews.jsonl')

    # Enrich
    texts_with_dois = list(
        examples[['sent_no_cit', 'source_doi']].itertuples(index=False, name=None))
    enriched_texts = enricher.enrich_batch(texts_with_dois)
    embeddings = embedder(enriched_texts)

    # Rank
    ranks = []
    for i in tqdm(range(len(examples))):
        embedding = embeddings[i]
        example = examples.iloc[i]
        ranks += get_ranks(
            example=example,
            embedding=embedding,
            table_name='lib',
            target_column=target_column,
            top_k=top_k,
            probes=probes
            )
    return ranks

In [36]:
results = ranks_at_k(
    examples=examples,
    embedder_name='BAAI/bge-small-en',
    enricher_name='identity',
    target_column='bge',
    top_k=7000,
    probes=80
)
print(len(results))

Embedding model: BAAI/bge-small-en, Enricher: identity


  0%|          | 0/100 [00:00<?, ?it/s]

Session resources set for query optimization:

                SET synchronous_commit = 'on';
                -- SET wal_level = 'replica';
                -- SET max_wal_size = 'DEFAULT';
                SET maintenance_work_mem = '1GB';
                -- SET random_page_cost = '1.1';
                -- SET parallel_tuple_cost = '0.1';
                -- SET parallel_setup_cost = '1000';
                SET max_parallel_workers = '60';
                SET work_mem = '1GB';
                SET max_parallel_workers_per_gather = '60';
                -- SET shared_buffers = '28GB';
                SET effective_cache_size = '86GB';
                -- SET effective_io_concurrency = '200';
            


  1%|          | 1/100 [00:00<00:28,  3.50it/s]

  Query execution time: 0.27 seconds
  Found 7000 results
top_k: 7000
Query time: 0.28s
Found 7000 results
Session resources set for query optimization:

                SET synchronous_commit = 'on';
                -- SET wal_level = 'replica';
                -- SET max_wal_size = 'DEFAULT';
                SET maintenance_work_mem = '1GB';
                -- SET random_page_cost = '1.1';
                -- SET parallel_tuple_cost = '0.1';
                -- SET parallel_setup_cost = '1000';
                SET max_parallel_workers = '60';
                SET work_mem = '1GB';
                SET max_parallel_workers_per_gather = '60';
                -- SET shared_buffers = '28GB';
                SET effective_cache_size = '86GB';
                -- SET effective_io_concurrency = '200';
            


...

  Query execution time: 0.32 seconds
  Found 7000 results
top_k: 7000
Query time: 0.33s
Found 7000 results
Session resources set for query optimization:

                SET synchronous_commit = 'on';
                -- SET wal_level = 'replica';
                -- SET max_wal_size = 'DEFAULT';
                SET maintenance_work_mem = '1GB';
                -- SET random_page_cost = '1.1';
                -- SET parallel_tuple_cost = '0.1';
                -- SET parallel_setup_cost = '1000';
                SET max_parallel_workers = '60';
                SET work_mem = '1GB';
                SET max_parallel_workers_per_gather = '60';
                -- SET shared_buffers = '28GB';
                SET effective_cache_size = '86GB';
                -- SET effective_io_concurrency = '200';
            


 99%|█████████▉| 99/100 [00:32<00:00,  3.10it/s]

  Query execution time: 0.25 seconds
  Found 7000 results
top_k: 7000
Query time: 0.26s
Found 7000 results
Session resources set for query optimization:

                SET synchronous_commit = 'on';
                -- SET wal_level = 'replica';
                -- SET max_wal_size = 'DEFAULT';
                SET maintenance_work_mem = '1GB';
                -- SET random_page_cost = '1.1';
                -- SET parallel_tuple_cost = '0.1';
                -- SET parallel_setup_cost = '1000';
                SET max_parallel_workers = '60';
                SET work_mem = '1GB';
                SET max_parallel_workers_per_gather = '60';
                -- SET shared_buffers = '28GB';
                SET effective_cache_size = '86GB';
                -- SET effective_io_concurrency = '200';
            


100%|██████████| 100/100 [00:33<00:00,  3.01it/s]

  Query execution time: 0.27 seconds
  Found 7000 results
top_k: 7000
Query time: 0.29s
Found 7000 results
111


In [37]:
found = len([r for r in results if r != -1])
print(f"Found {found} out of {len(results)}")
print(f"Found {found/len(results)*100:.2f}%")

Found 92 out of 111
Found 82.88%


In [35]:
for i, result in enumerate(results):
    print(i, result)

0 -1
1 596
2 7281
3 1972
4 0
5 24
6 0
7 503
8 -1
9 468
10 -1
11 -1
12 5
13 1
14 -1
15 20
16 1272
17 164
18 8045
19 3530
20 571
21 365
22 0
23 1
24 0
25 -1
26 5
27 177
28 3
29 -1
30 1
31 32
32 660
33 287
34 77
35 1
36 3891
37 95
38 340
39 0
40 104
41 3
42 1
43 -1
44 2904
45 3100
46 -1
47 0
48 26
49 47
50 -1
51 1
52 5571
53 452
54 250
55 1
56 2
57 0
58 383
59 -1
60 0
61 -1
62 0
63 329
64 -1
65 97
66 319
67 339
68 46
69 6
70 1746
71 0
72 70
73 6
74 -1
75 -1
76 3615
77 -1
78 9
79 2704
80 4
81 -1
82 1
83 22
84 565
85 531
86 1150
87 4
88 -1
89 2098
90 -1
91 30
92 16
93 6665
94 1
95 7
96 750
97 540
98 204
99 25
100 221
101 0
102 1344
103 607
104 -1
105 1
106 54
107 1583
108 688
109 2
110 27


In [24]:
examples.iloc[93]['sent_no_cit']

'This is clearly shown by the lateral duplicate sampling performed by , SI fig. S4) .'

In [4]:
from itertools import product
                      
embedding_models = ['BAAI/bge-small-en']
enrichment_methods = ['identity', 'add_abstract',
                      'add_title', 'add_title_and_abstract']
if device == 'cuda':
    embedding_models += ['bert-base-uncased', 'adsabs/astroBERT']

combos = list(product(embedding_models, enrichment_methods))
print(f"Combos: {combos}")

Combos: [('BAAI/bge-small-en', 'identity'), ('BAAI/bge-small-en', 'add_abstract'), ('BAAI/bge-small-en', 'add_title'), ('BAAI/bge-small-en', 'add_title_and_abstract')]


In [ ]:
rank_data = {}
for enricher_name in ['identity', 'add_abstract', 'add_title', 'add_title_and_abstract']:
    rank_data[enricher_name] = ranks_at_k(
        examples=examples,
        embedder_name='BAAI/bge-small-en',
        enricher_name=enricher_name,
        target_column='bge_norm',
        top_k=300_000,
        probes=20
    )

Embedding model: BAAI/bge-small-en, Enricher: identity


  0%|          | 0/1 [00:00<?, ?it/s]

  Query execution time: 37.84 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


100%|██████████| 1/1 [00:48<00:00, 48.30s/it]


Embedding model: BAAI/bge-small-en, Enricher: add_abstract


...

100%|██████████| 1/1 [00:46<00:00, 46.11s/it]


Embedding model: BAAI/bge-small-en, Enricher: add_title_and_abstract


  0%|          | 0/1 [00:00<?, ?it/s]

  Query execution time: 34.77 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


100%|██████████| 1/1 [00:44<00:00, 44.96s/it]


In [6]:
print(examples.iloc[0].sent_original)
for enricher_name, ranks in rank_data.items():
    print(f"Enricher: {enricher_name}. Rank: {ranks[0]}")

CGRO observations of X-ray binaries detected non-thermal power-law tails extending well beyond 100 keV with a photon index Γ LE ≈2.5–3 (Grove et al. 1998 ). 
Enricher: identity. Rank: 21
Enricher: add_abstract. Rank: 348
Enricher: add_title. Rank: 1
Enricher: add_title_and_abstract. Rank: 285


In [ ]:
new_examples = data.sample(2, random_state=29)

new_rank_data = {}
for enricher_name in ['identity', 'add_abstract', 'add_title', 'add_title_and_abstract']:
    new_rank_data[enricher_name] = ranks_at_k(
        examples=new_examples,
        embedder_name='BAAI/bge-small-en',
        enricher_name=enricher_name,
        target_column='bge_norm',
        top_k=2_261_334,
        ef_search=20
    )

Embedding model: BAAI/bge-small-en, Enricher: identity


  0%|          | 0/2 [00:00<?, ?it/s]

  Query execution time: 34.42 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


 50%|█████     | 1/2 [00:44<00:44, 44.23s/it]

  Query execution time: 34.71 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


...

  Query execution time: 34.30 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


 50%|█████     | 1/2 [00:43<00:43, 43.56s/it]

  Query execution time: 35.42 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


100%|██████████| 2/2 [01:28<00:00, 44.34s/it]

Enricher: identity. Rank: 1159
Enricher: add_abstract. Rank: 164
Enricher: add_title. Rank: 138
Enricher: add_title_and_abstract. Rank: 229


In [8]:
for enricher_name, ranks in new_rank_data.items():
    print(f"Enricher: {enricher_name}. Rank: {ranks}")

Enricher: identity. Rank: [1159, 288, 288]
Enricher: add_abstract. Rank: [164, 2903, 2903]
Enricher: add_title. Rank: [138, 13693, 13693]
Enricher: add_title_and_abstract. Rank: [229, 4420, 4420]


In [9]:
print(new_examples.iloc[1].sent_original)
print(new_examples.iloc[1].citation_dois)

Both Hood et al. ( 2009 ) and MacTaggart and Hood ( 2009 ) reported that a second MFR formed via magnetic reconnection (see below for a detailed description of this process) underneath the bodily emerged MFR. 
['10.1051/0004-6361/200912189', '10.1051/0004-6361/200912189']


In [10]:
more_examples = data.sample(4, random_state=1)
more_rank_data = {}
for enricher_name in ['identity', 'add_abstract', 'add_title', 'add_title_and_abstract']:
    more_rank_data[enricher_name] = ranks_at_k(
        examples=more_examples,
        embedder_name='BAAI/bge-small-en',
        enricher_name=enricher_name,
        target_column='bge_norm',
        top_k=2_261_334,
        ef_search=20
    )

Embedding model: BAAI/bge-small-en, Enricher: identity


  0%|          | 0/4 [00:00<?, ?it/s]

  Query execution time: 34.51 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


 25%|██▌       | 1/4 [00:44<02:14, 44.79s/it]

  Query execution time: 35.54 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


...

 50%|█████     | 2/4 [01:31<01:31, 45.72s/it]

  Query execution time: 35.81 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


 75%|███████▌  | 3/4 [02:17<00:45, 45.68s/it]

  Query execution time: 35.65 seconds
Found 2261334 results
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND
FOUND


100%|██████████| 4/4 [03:02<00:00, 45.58s/it]


In [ ]:
for enricher_name, ranks in new_rank_data.items():
    print(f"Enricher: {enricher_name}. Rank: {ranks}")

In [ ]:
# write rank_data dict to file
import json
with open('data/rank_data.json', 'w') as f:
    json.dump(rank_data, f)
print('Rank data saved to file')

In [ ]:
max(rank_data['identity']), max(rank_data['add_abstract']), max(rank_data['add_title'])

In [ ]:
research = pd.read_json('data/preprocessed/research.jsonl', lines=True)
research[research.doi == '10.1093/mnras/stab3351']

In [ ]:
model_names_to_filenames = {
    'BAAI/bge-small-en': 'bge',
    'bert-base-uncased': 'bert',
    'adsabs/astroBERT': 'astrobert'
}